In [20]:
import pandas as pd
import numpy as np
from numpy import ones,vstack
from numpy.linalg import lstsq
import math
import csv
import geopy.distance
import geopy.point

In [21]:
mapped_links_csv_file = "mapped_links.csv"
link_ids_near_precip_csv = "/home/gayan/Desktop/CMLARE/Data/mapped_links_near_"
mapped_links_field_names = ["cell_tower_id1","cell_tower_id2","source1_name","source2_name", "district1","district2",
                            "latitude1","latitude2","longitude1","longitude2","nearest_precip_station","distance",
                            "nearest_precip_lat","nearest_precip_long","id_1","id_2"]
formatted_links_field_names = ["cell_tower_id1","cell_tower_id2","source1_name","source2_name", "district1","district2",
                            "latitude1","latitude2","longitude1","longitude2","nearest_precip_station","distance",
                            "nearest_precip_lat","nearest_precip_long","id_1","id_2","PathLength"]
mapped_links_near_precip = []
precip_devices_csv_file = "precipitaion_stations.csv"
precip_devices_field_names = ["name","lat","long","threshold"]
# maximum_perpendicular_distance = 10
link_ids_near_precip_csv = link_ids_near_precip_csv + ".csv"

In [22]:
def calc_equation(lat1, lang1, lat2, lang2):
    points = [(lat1, lang1), (lat2,lang2)]
    x_coords, y_coords = zip(*points)
    A = vstack([x_coords, ones(len(x_coords))]).T
    m, c = lstsq(A, y_coords)[0]
    #print("Line Solution is y = {m}x + {c}".format(m=m, c=c))
    return m,c

def calc_distance(lat1, lang1, lat2, lang2,lat_precip,long_precip):
#     m,c = calc_equation(lat1, lang1, lat2, lang2)
#     long = np.array([long_precip])
#     lat = np.array([lat_precip])
#     distance_function = lambda lat,long : abs(lat*m - long + c)/math.sqrt(lat**2 + long**2)
#     vec_distance_function = np.vectorize(distance_function)
#     distances = vec_distance_function(lat,long)*1000000
#     index_of_minimum_distance_precip = np.argmin(distances)
    lat1= float(lat1)
    lat2 = float(lat2)
    lang1 = float(lang1)
    lang2 = float(lang2)
    start_cord = (lat_precip,long_precip)
    mid_point = ((lat1+lat2)/2,(lang1+lang2)/2)
    print(start_cord)
    print(mid_point)
    distance = geopy.distance.vincenty(start_cord, mid_point).km
    print(distance)
    return distance
def path_distance(lat1,lang1,lat2,lang2):
    lat1= float(lat1)
    lat2 = float(lat2)
    lang1 = float(lang1)
    lang2 = float(lang2)
    start = (lat1,lang1)
    end = (lat2,lang2)
    return geopy.distance.vincenty(start, end).km    

In [23]:
precip_devices = []
with open(precip_devices_csv_file,"r") as precip_devices_csv:
    precip_devices_dict = csv.DictReader(precip_devices_csv,fieldnames=precip_devices_field_names)
    next(precip_devices_dict)
    for precip_device in precip_devices_dict:
        prep = {}
        prep['name'] = precip_device['name']
        prep['lat'] = float(precip_device['lat'])
        prep['long'] = float(precip_device['long'])
        prep["threshold"] = float(precip_device["threshold"])
        precip_devices.append(prep)
with open(mapped_links_csv_file,"r") as mapped_links_csv:
    mapped_links = csv.DictReader(mapped_links_csv, fieldnames=mapped_links_field_names)
    next(mapped_links)
#     print(precip_devices)
    for link in mapped_links:
        for precip_device in precip_devices:
            distance = calc_distance(link["latitude1"],link["longitude1"],link["latitude2"],link["longitude2"],precip_device["lat"],precip_device["long"])
            if(distance<=precip_device["threshold"]):
#                 print(precip_device["threshold"])
#                 print("***************")
                link["distance"] = distance
                link["nearest_precip_station"] = precip_device["name"]
                link["nearest_precip_lat"] = precip_device["lat"]
                link["nearest_precip_long"] = precip_device["long"]
                link["PathLength"] = path_distance(link["latitude1"],link["longitude1"],link["latitude2"],link["longitude2"])
                mapped_links_near_precip.append(link)
                break
    with open(link_ids_near_precip_csv,"w") as processed_file_csv:
        dictionary_writer = csv.DictWriter(processed_file_csv,formatted_links_field_names)
        dictionary_writer.writeheader()
        mapped_links_near_precip = [dict(t) for t in set([tuple(d.items()) for d in mapped_links_near_precip])]
        mapped_links_near_precip_sorted = sorted(mapped_links_near_precip , key=lambda k: k['nearest_precip_station'])
        dictionary_writer.writerows(mapped_links_near_precip_sorted)

(6.904511, 80.407853)
(9.2707405, 80.78805)
265.0298646582301
(6.865576, 79.958181)
(9.2707405, 80.78805)
281.2882880125646
(6.943741, 79.87887)
(9.2707405, 80.78805)
276.17504873915425
(6.908296, 79.91918)
(9.2707405, 80.78805)
278.27308058862246
(6.906769, 80.118111)
(9.2707405, 80.78805)
271.6725495430239
(7.169737, 80.258512)
(9.2707405, 80.78805)
239.57738444397634
(6.904511, 80.407853)
(8.948889, 80.7344445)
228.94849011426754
(6.865576, 79.958181)
(8.948889, 80.7344445)
245.78935196737226
(6.943741, 79.87887)
(8.948889, 80.7344445)
240.9888474833337
(6.908296, 79.91918)
(8.948889, 80.7344445)
242.92285398464148
(6.906769, 80.118111)
(8.948889, 80.7344445)
235.85113547067272
(7.169737, 80.258512)
(8.948889, 80.7344445)
203.64017081590788
(6.904511, 80.407853)
(8.948889, 80.7344445)
228.94849011426754
(6.865576, 79.958181)
(8.948889, 80.7344445)
245.78935196737226
(6.943741, 79.87887)
(8.948889, 80.7344445)
240.9888474833337
(6.908296, 79.91918)
(8.948889, 80.7344445)
242.92285398

157.36436904166462
(6.865576, 79.958181)
(8.157074999999999, 79.73134999999999)
145.00855003776763
(6.943741, 79.87887)
(8.157074999999999, 79.73134999999999)
135.17081627287968
(6.908296, 79.91918)
(8.157074999999999, 79.73134999999999)
139.6537727371297
(6.906769, 80.118111)
(8.157074999999999, 79.73134999999999)
144.71376585320718
(7.169737, 80.258512)
(8.157074999999999, 79.73134999999999)
123.71775251188043
(6.904511, 80.407853)
(8.155416500000001, 80.2806945)
139.05185745385288
(6.865576, 79.958181)
(8.155416500000001, 80.2806945)
147.02174302573607
(6.943741, 79.87887)
(8.155416500000001, 80.2806945)
141.1502072470522
(6.908296, 79.91918)
(8.155416500000001, 80.2806945)
143.5781127165221
(6.906769, 80.118111)
(8.155416500000001, 80.2806945)
139.2530299737992
(7.169737, 80.258512)
(8.155416500000001, 80.2806945)
109.0377542985744
(6.904511, 80.407853)
(7.072331, 79.895478)
59.58071234546808
(6.865576, 79.958181)
(7.072331, 79.895478)
23.89193396383493
(6.943741, 79.87887)
(7.0723

226.29374033502825
(7.169737, 80.258512)
(8.7220835, 81.064861)
193.3358098522283
(6.904511, 80.407853)
(8.7220835, 81.064861)
213.67542027568217
(6.865576, 79.958181)
(8.7220835, 81.064861)
238.861263879964
(6.943741, 79.87887)
(8.7220835, 81.064861)
236.19610949638115
(6.908296, 79.91918)
(8.7220835, 81.064861)
237.07386501431253
(6.906769, 80.118111)
(8.7220835, 81.064861)
226.29374033502825
(7.169737, 80.258512)
(8.7220835, 81.064861)
193.3358098522283
(6.904511, 80.407853)
(7.592035, 79.8241)
99.68568487535893
(6.865576, 79.958181)
(7.592035, 79.8241)
81.69371391462424
(6.943741, 79.87887)
(7.592035, 79.8241)
71.95083216440344
(6.908296, 79.91918)
(7.592035, 79.8241)
76.34159515241106
(6.906769, 80.118111)
(7.592035, 79.8241)
82.44751863248425
(7.169737, 80.258512)
(7.592035, 79.8241)
66.94303505213993
(6.904511, 80.407853)
(7.592035, 79.8241)
99.68568487535893
(6.865576, 79.958181)
(7.592035, 79.8241)
81.69371391462424
(6.943741, 79.87887)
(7.592035, 79.8241)
71.95083216440344
(6

(7.169737, 80.258512)
(7.2368055, 79.924625)
37.61525900546159
(6.904511, 80.407853)
(8.499166500000001, 80.738611)
180.09496343016372
(6.865576, 79.958181)
(8.499166500000001, 80.738611)
200.13288169756623
(6.943741, 79.87887)
(8.499166500000001, 80.738611)
196.43349984161767
(6.908296, 79.91918)
(8.499166500000001, 80.738611)
197.80560551100183
(6.906769, 80.118111)
(8.499166500000001, 80.738611)
188.9456794940012
(7.169737, 80.258512)
(8.499166500000001, 80.738611)
156.27135651535022
(6.904511, 80.407853)
(8.499166500000001, 80.738611)
180.09496343016372
(6.865576, 79.958181)
(8.499166500000001, 80.738611)
200.13288169756623
(6.943741, 79.87887)
(8.499166500000001, 80.738611)
196.43349984161767
(6.908296, 79.91918)
(8.499166500000001, 80.738611)
197.80560551100183
(6.906769, 80.118111)
(8.499166500000001, 80.738611)
188.9456794940012
(7.169737, 80.258512)
(8.499166500000001, 80.738611)
156.27135651535022
(6.904511, 80.407853)
(6.9036, 79.9174)
54.20379948516685
(6.865576, 79.958181)

89.41549024994846
(6.943741, 79.87887)
(7.5049865, 80.4536445)
88.78138536965744
(6.908296, 79.91918)
(7.5049865, 80.4536445)
88.53809211610462
(6.906769, 80.118111)
(7.5049865, 80.4536445)
75.82994269469593
(7.169737, 80.258512)
(7.5049865, 80.4536445)
42.88162634090731
(6.904511, 80.407853)
(6.797021, 79.8793335)
59.61443466425029
(6.865576, 79.958181)
(6.797021, 79.8793335)
11.551478041749656
(6.943741, 79.87887)
(6.797021, 79.8793335)
16.22587018066779
(6.908296, 79.91918)
(6.797021, 79.8793335)
13.070293166600683
(6.906769, 80.118111)
(6.797021, 79.8793335)
29.04901764694758
(7.169737, 80.258512)
(6.797021, 79.8793335)
58.77508246366154
(6.904511, 80.407853)
(6.797021, 79.8793335)
59.61443466425029
(6.865576, 79.958181)
(6.797021, 79.8793335)
11.551478041749656
(6.943741, 79.87887)
(6.797021, 79.8793335)
16.22587018066779
(6.908296, 79.91918)
(6.797021, 79.8793335)
13.070293166600683
(6.906769, 80.118111)
(6.797021, 79.8793335)
29.04901764694758
(7.169737, 80.258512)
(6.797021, 79

...

(6.8805555, 79.875833)
9.251028615636196
(6.943741, 79.87887)
(6.8805555, 79.875833)
6.995763222785396
(6.908296, 79.91918)
(6.8805555, 79.875833)
5.688798793198348
(6.904511, 80.407853)
(6.8805555, 79.875833)
58.85866449691772
(6.865576, 79.958181)
(6.8805555, 79.875833)
9.251028615636196
(6.943741, 79.87887)
(6.8805555, 79.875833)
6.995763222785396
(6.908296, 79.91918)
(6.8805555, 79.875833)
5.688798793198348
(6.904511, 80.407853)
(7.109204, 80.15863350000001)
35.647378863486125
(6.865576, 79.958181)
(7.109204, 80.15863350000001)
34.878798915591645
(6.943741, 79.87887)
(7.109204, 80.15863350000001)
35.92100232542864
(6.908296, 79.91918)
(7.109204, 80.15863350000001)
34.549803654573886
(6.906769, 80.118111)
(7.109204, 80.15863350000001)
22.830803907165617
(7.169737, 80.258512)
(7.109204, 80.15863350000001)
12.904952831815468
(6.904511, 80.407853)
(6.923438, 79.8720055)
59.25634394800376
(6.865576, 79.958181)
(6.923438, 79.8720055)
11.474119559354039
(6.943741, 79.87887)
(6.923438, 79.

(7.49291, 80.41024999999999)
72.40873018567446
(7.169737, 80.258512)
(7.49291, 80.41024999999999)
39.47257227219307
(6.904511, 80.407853)
(8.074152, 80.6859915)
132.94730409670186
(6.865576, 79.958181)
(8.074152, 80.6859915)
155.94463693404802
(6.943741, 79.87887)
(8.074152, 80.6859915)
153.50721261876188
(6.908296, 79.91918)
(8.074152, 80.6859915)
154.2324130561998
(6.906769, 80.118111)
(8.074152, 80.6859915)
143.5154138000159
(7.169737, 80.258512)
(8.074152, 80.6859915)
110.58672714320846
(6.904511, 80.407853)
(6.929851, 79.842389)
62.55482133276761
(6.865576, 79.958181)
(6.929851, 79.842389)
14.638832925668483
(6.943741, 79.87887)
(6.929851, 79.842389)
4.314247513651136
(6.904511, 80.407853)
(6.9364725, 79.90551400000001)
55.627809964276196
(6.865576, 79.958181)
(6.9364725, 79.90551400000001)
9.764890861375292
(6.943741, 79.87887)
(6.9364725, 79.90551400000001)
3.05215982056812
(6.904511, 80.407853)
(7.04862, 80.01570000000001)
46.170941818072556
(6.865576, 79.958181)
(7.04862, 80.0

(6.865576, 79.958181)
(6.8951285, 79.94149999999999)
3.752364439960927
(6.904511, 80.407853)
(7.713566500000001, 80.32844449999999)
89.90388707300343
(6.865576, 79.958181)
(7.713566500000001, 80.32844449999999)
102.30630479928875
(6.943741, 79.87887)
(7.713566500000001, 80.32844449999999)
98.55148614374542
(6.908296, 79.91918)
(7.713566500000001, 80.32844449999999)
99.866424543573
(6.906769, 80.118111)
(7.713566500000001, 80.32844449999999)
92.19870427860924
(7.169737, 80.258512)
(7.713566500000001, 80.32844449999999)
60.637090314644375
(6.904511, 80.407853)
(6.9452, 79.89750000000001)
56.57975862649803
(6.865576, 79.958181)
(6.9452, 79.89750000000001)
11.068597592485125
(6.943741, 79.87887)
(6.9452, 79.89750000000001)
2.0650815464095955
(6.904511, 80.407853)
(6.9341645, 79.8451445)
62.2736218318755
(6.865576, 79.958181)
(6.9341645, 79.8451445)
14.61511430883444
(6.943741, 79.87887)
(6.9341645, 79.8451445)
3.874541928254068
(6.904511, 80.407853)
(6.9341645, 79.8451445)
62.2736218318755

(7.939978, 79.850811)
110.22029896283667
(6.908296, 79.91918)
(7.939978, 79.850811)
114.34599736388977
(6.906769, 80.118111)
(7.939978, 79.850811)
118.01399739143383
(7.169737, 80.258512)
(7.939978, 79.850811)
96.33614874921861
(6.904511, 80.407853)
(7.089373, 80.015761)
47.90590114872227
(6.865576, 79.958181)
(7.089373, 80.015761)
25.554615933866884
(6.943741, 79.87887)
(7.089373, 80.015761)
22.094408118332357
(6.908296, 79.91918)
(7.089373, 80.015761)
22.69151575010053
(6.906769, 80.118111)
(7.089373, 80.015761)
23.145388334639385
(7.169737, 80.258512)
(7.089373, 80.015761)
28.249818470666728
(6.904511, 80.407853)
(9.67472, 79.93254999999999)
310.8196471968939
(6.865576, 79.958181)
(9.67472, 79.93254999999999)
310.69703812530946
(6.943741, 79.87887)
(9.67472, 79.93254999999999)
302.09778413793515
(6.908296, 79.91918)
(9.67472, 79.93254999999999)
305.96333811057303
(6.906769, 80.118111)
(9.67472, 79.93254999999999)
306.810288345157
(7.169737, 80.258512)
(9.67472, 79.93254999999999)
27

(7.43032, 80.00125)
58.438819300252696
(6.906769, 80.118111)
(7.43032, 80.00125)
59.32167186415211
(7.169737, 80.258512)
(7.43032, 80.00125)
40.46600756513908
(6.904511, 80.407853)
(6.898142, 79.8741835)
58.98443415245136
(6.865576, 79.958181)
(6.898142, 79.8741835)
9.95774003249549
(6.943741, 79.87887)
(6.898142, 79.8741835)
5.069338529487691
(6.908296, 79.91918)
(6.898142, 79.8741835)
5.0981243406643015
(6.904511, 80.407853)
(6.898142, 79.8741835)
58.98443415245136
(6.865576, 79.958181)
(6.898142, 79.8741835)
9.95774003249549
(6.943741, 79.87887)
(6.898142, 79.8741835)
5.069338529487691
(6.908296, 79.91918)
(6.898142, 79.8741835)
5.0981243406643015
(6.904511, 80.407853)
(6.898142, 79.8741835)
58.98443415245136
(6.865576, 79.958181)
(6.898142, 79.8741835)
9.95774003249549
(6.943741, 79.87887)
(6.898142, 79.8741835)
5.069338529487691
(6.908296, 79.91918)
(6.898142, 79.8741835)
5.0981243406643015
(6.904511, 80.407853)
(6.8793055, 79.8865275)
57.68451775862506
(6.865576, 79.958181)
(6.87